# Chef AI com LangGraph e APIs públicas

Agente especialista em criar receitas a partir de uma lista de ingredientes e da quantidade de receitas desejada. Usa TheMealDB e TheCocktailDB como ferramentas de pesquisa e retorna JSON com: nome, ingredientes (com porções), modo de fazer e rendimento.

In [9]:
from dotenv import load_dotenv
_ = load_dotenv()

In [10]:
from typing import TypedDict, Annotated, List, Optional
import operator
import json
from urllib.parse import urlencode
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError

from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from pydantic import BaseModel, Field

In [11]:
class ItemIngrediente(BaseModel):
    nome: str = Field(..., description="Nome do ingrediente")
    quantidade: str = Field(..., description="Quantidade e unidade, ex: '2 xícaras', '200 g'")

class ReceitaBase(BaseModel):
    nome: str = Field(..., description="Nome da receita")
    ingredientes: List[ItemIngrediente] = Field(..., description="Lista de ingredientes")
    modo_de_fazer: List[str] = Field(..., description="Passos em pt-BR")
    rendimento: str = Field(..., description="Ex.: 'serve 4 porções'")

class RespostaReceitasBase(BaseModel):
    receitas: List[ReceitaBase]

class DrinkSugestao(BaseModel):
    nome: str = Field(..., description="Nome do drink")
    ingredientes: List[ItemIngrediente] = Field(default_factory=list, description="Ingredientes do drink com porções")

class SugestaoDrinksPorReceita(BaseModel):
    receita: str = Field(..., description="Nome da receita a harmonizar")
    drinks: List[str] = Field(..., min_length=3, max_length=3, description="Lista de drinks sugeridos")

class SugestoesDrinks(BaseModel):
    sugestoes: List[SugestaoDrinksPorReceita]

class Receita(ReceitaBase):
    sugestoes_drinks: List[DrinkSugestao] = Field(
        ..., 
        alias="Sugestões de drinks para acompanhar", 
        description="Exatamente 3 drinks que harmonizam com a receita"
    )
    model_config = {"populate_by_name": True}

class RespostaReceitas(BaseModel):
    receitas: List[Receita]

In [12]:
def _http_get_json(url: str) -> dict:
    """Faz requisição HTTP e retorna JSON com tratamento de erros."""
    try:
        req = Request(url, headers={"User-Agent": "chef-ai/1.0"})
        with urlopen(req, timeout=20) as r:
            data = r.read().decode("utf-8")
        return json.loads(data)
    except (URLError, HTTPError, json.JSONDecodeError) as e:
        print(f"Erro ao acessar {url}: {e}")
        return {}

def _parse_meal_detail(m: dict) -> dict:
    """Extrai detalhes de uma receita do TheMealDB."""
    if not m:
        return {}
    ings = []
    for i in range(1, 21):
        ing = (m.get(f"strIngredient{i}") or "").strip()
        mea = (m.get(f"strMeasure{i}") or "").strip()
        if ing:
            ings.append({"nome": ing, "quantidade": mea or "a gosto"})
    return {
        "nome": m.get("strMeal", "Receita sem nome"),
        "categoria": m.get("strCategory", ""),
        "cozinha": m.get("strArea", ""),
        "ingredientes": ings,
        "instrucoes": (m.get("strInstructions") or "").strip()[:1200]
    }

def _parse_drink_detail(d: dict) -> dict:
    """Extrai detalhes de um drink do TheCocktailDB."""
    if not d:
        return {}
    ings = []
    for i in range(1, 16):
        ing = (d.get(f"strIngredient{i}") or "").strip()
        mea = (d.get(f"strMeasure{i}") or "").strip()
        if ing:
            ings.append({"nome": ing, "quantidade": mea or "a gosto"})
    return {
        "nome": d.get("strDrink", "Drink sem nome"),
        "categoria": d.get("strCategory", ""),
        "alcoholic": d.get("strAlcoholic", ""),
        "ingredientes": ings,
        "instrucoes": (d.get("strInstructions") or "").strip()[:800]
    }

def _build_drink_sugestao(name: str) -> DrinkSugestao:
    """Busca drink na API e retorna objeto DrinkSugestao."""
    try:
        data = _http_get_json(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?{urlencode({'s': name})}")
        drinks = data.get("drinks") or []
        alvo = next((d for d in drinks if (d.get("strDrink") or "").lower() == name.lower()), drinks[0] if drinks else None)
        
        if not alvo:
            return DrinkSugestao(
                nome=name,
                ingredientes=[ItemIngrediente(nome="Informação indisponível", quantidade="Consulte um bartender")]
            )
        
        detail = _parse_drink_detail(alvo)
        itens = [
            ItemIngrediente(
                nome=ing.get("nome", "Ingrediente"),
                quantidade=ing.get("quantidade", "a gosto")
            )
            for ing in detail.get("ingredientes", []) if ing.get("nome")
        ]
        
        if not itens:
            itens = [ItemIngrediente(nome="Ingrediente não especificado", quantidade="a gosto")]
        
        return DrinkSugestao(nome=detail.get("nome", name), ingredientes=itens)
    except Exception as e:
        print(f"Erro ao buscar drink '{name}': {e}")
        return DrinkSugestao(
            nome=name,
            ingredientes=[ItemIngrediente(nome="Erro ao buscar", quantidade="Tente novamente")]
        )

@tool("mealdb_search")
def mealdb_search(query: str) -> str:
    """Busca receitas de comidas no TheMealDB pelo termo informado. Retorna até 3 sugestões resumidas."""
    base = "https://www.themealdb.com/api/json/v1/1/search.php"
    url = f"{base}?{urlencode({'s': query})}"
    data = _http_get_json(url)
    meals = data.get("meals") or []
    results = [_parse_meal_detail(m) for m in meals[:3] if m]
    return json.dumps({"fonte": "TheMealDB", "query": query, "sugestoes": results}, ensure_ascii=False)

@tool("mealdb_filter_ingredient")
def mealdb_filter_ingredient(ingredient: str, max_results: int = 3) -> str:
    """Filtra comidas por ingrediente no TheMealDB e retorna detalhes (até max_results)."""
    filt = f"https://www.themealdb.com/api/json/v1/1/filter.php?{urlencode({'i': ingredient})}"
    data = _http_get_json(filt)
    meals = data.get("meals") or []
    results = []
    for m in meals[:max_results]:
        _id = m.get("idMeal")
        if not _id:
            continue
        detail = _http_get_json(f"https://www.themealdb.com/api/json/v1/1/lookup.php?i={_id}")
        det = (detail.get("meals") or [None])[0]
        if det:
            results.append(_parse_meal_detail(det))
    return json.dumps({"fonte": "TheMealDB", "ingredient": ingredient, "sugestoes": results}, ensure_ascii=False)

@tool("cocktaildb_search")
def cocktaildb_search(query: str) -> str:
    """Busca drinks no TheCocktailDB pelo termo informado. Retorna até 3 sugestões resumidas."""
    base = "https://www.thecocktaildb.com/api/json/v1/1/search.php"
    url = f"{base}?{urlencode({'s': query})}"
    data = _http_get_json(url)
    drinks = data.get("drinks") or []
    results = [_parse_drink_detail(d) for d in drinks[:3] if d]
    payload = {"fonte": "TheCocktailDB", "query": query, "sugestoes": results}
    if not results:
        payload["debug"] = "Nenhuma sugestão retornada pela API."
    return json.dumps(payload, ensure_ascii=False)

@tool("cocktaildb_filter_ingredient")
def cocktaildb_filter_ingredient(ingredient: str, max_results: int = 3) -> str:
    """Filtra drinks por ingrediente no TheCocktailDB e retorna detalhes (até max_results)."""
    filt = f"https://www.thecocktaildb.com/api/json/v1/1/filter.php?{urlencode({'i': ingredient})}"
    data = _http_get_json(filt)
    drinks = data.get("drinks") or []
    results = []
    for d in drinks[:max_results]:
        _id = d.get("idDrink")
        if not _id:
            continue
        detail = _http_get_json(f"https://www.thecocktaildb.com/api/json/v1/1/lookup.php?i={_id}")
        det = (detail.get("drinks") or [None])[0]
        if det:
            results.append(_parse_drink_detail(det))
    payload = {"fonte": "TheCocktailDB", "ingredient": ingredient, "sugestoes": results}
    if not results:
        payload["debug"] = "Nenhuma sugestão retornada pela API."
    return json.dumps(payload, ensure_ascii=False)

In [13]:
# Define o estado do agente para o LangGraph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [14]:
class RecipeAgentGraph:
    def __init__(self, model_tools: str = "gpt-4o-mini", model_format: str = "gpt-4o-mini"):
        self.system_tools = (
            "Você é um chef profissional. Pesquise inspirações usando as ferramentas quando útil: "
            "mealdb_search / mealdb_filter_ingredient para comidas, "
            "cocktaildb_search / cocktaildb_filter_ingredient para drinks. "
            "Mantenha foco nos ingredientes do usuário e colete evidências para os passos e harmonizações. "
            "Não gere a saída final aqui; apenas junte evidências para a próxima etapa."
        )
        self.system_format = (
            "Você deve produzir exclusivamente o objeto JSON no esquema RespostaReceitasBase, "
            "contendo receitas completas com nome, ingredientes (com quantidade e unidade), "
            "modo de fazer (lista de passos) e rendimento. Escreva em pt-BR e use os ingredientes fornecidos."
        )
        self.system_drinks = (
            "Você receberá receitas em JSON. Para cada receita, retorne exatamente 3 drinks plausíveis "
            "que harmonizem bem, preferindo opções distintas e conhecidas."
        )

        self.tools = {t.name: t for t in [mealdb_search, mealdb_filter_ingredient, cocktaildb_search, cocktaildb_filter_ingredient]}
        base_model = ChatOpenAI(model=model_tools)
        self.model_tools = base_model.bind_tools(list(self.tools.values()))
        self.model_format_base = ChatOpenAI(model=model_format).with_structured_output(RespostaReceitasBase)
        self.model_drinks = ChatOpenAI(model=model_format).with_structured_output(SugestoesDrinks)

        graph = StateGraph(AgentState)
        graph.add_node("llm", self._llm)
        graph.add_node("action", self._take_action)
        graph.add_node("format_recipes", self._format_recipes)
        graph.add_node("suggest_drinks", self._suggest_drinks)
        graph.add_node("hydrate_drinks", self._hydrate_drinks)
        graph.add_conditional_edges("llm", self._has_action, {True: "action", False: "format_recipes"})
        graph.add_edge("action", "llm")
        graph.add_edge("format_recipes", "suggest_drinks")
        graph.add_edge("suggest_drinks", "hydrate_drinks")
        graph.add_edge("hydrate_drinks", END)
        graph.set_entry_point("llm")
        self.graph = graph.compile()

    def _has_action(self, state: AgentState) -> bool:
        """Verifica se a última mensagem contém chamadas de ferramentas."""
        result = state["messages"][-1]
        return len(getattr(result, "tool_calls", []) or []) > 0

    def _llm(self, state: AgentState) -> dict:
        """Invoca o LLM com ferramentas vinculadas."""
        mensagens = [SystemMessage(content=self.system_tools)] + state["messages"]
        msg = self.model_tools.invoke(mensagens)
        return {"messages": [msg]}

    def _normalize_tool_args(self, name: str, raw_args) -> dict:
        """Normaliza argumentos de tool calls para dict consistente."""
        primary_keys = {
            "mealdb_search": "query",
            "cocktaildb_search": "query",
            "mealdb_filter_ingredient": "ingredient",
            "cocktaildb_filter_ingredient": "ingredient",
        }
        if raw_args is None:
            return {}
        if isinstance(raw_args, dict):
            return raw_args
        if isinstance(raw_args, str):
            try:
                loaded = json.loads(raw_args)
                return self._normalize_tool_args(name, loaded)
            except Exception:
                key = primary_keys.get(name, "input")
                return {key: raw_args}
        if isinstance(raw_args, (list, tuple)) and raw_args:
            key = primary_keys.get(name, "input")
            return {key: raw_args[0]} if len(raw_args) == 1 else {key: list(raw_args)}
        key = primary_keys.get(name, "input")
        return {key: str(raw_args)}

    def _take_action(self, state: AgentState) -> dict:
        """Executa chamadas de ferramentas e retorna ToolMessages."""
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for call in tool_calls:
            name = call["name"]
            args = self._normalize_tool_args(name, call.get("args"))
            if name not in self.tools:
                out = json.dumps({"erro": "bad tool name, retry"}, ensure_ascii=False)
            else:
                try:
                    out = self.tools[name].invoke(args)
                except Exception as e:
                    out = json.dumps({"erro": f"Falha ao executar {name}: {str(e)}"}, ensure_ascii=False)
            results.append(ToolMessage(tool_call_id=call["id"], name=name, content=str(out)))
        return {"messages": results}

    def _format_recipes(self, state: AgentState) -> dict:
        """Gera as receitas base em JSON estruturado."""
        messages = [SystemMessage(content=self.system_format)] + state["messages"]
        obj = self.model_format_base.invoke(messages)
        return {"messages": [AIMessage(content=obj.model_dump_json(ensure_ascii=False, indent=2))]}

    def _suggest_drinks(self, state: AgentState) -> dict:
        """Sugere 3 drinks por receita baseado nos ingredientes."""
        base_json = state["messages"][-1].content
        base = RespostaReceitasBase.model_validate_json(base_json)
        resumo = []
        for receita in base.receitas:
            principais = ", ".join(ing.nome for ing in receita.ingredientes[:5]) or "ingredientes variados"
            resumo.append(f"- {receita.nome}: ingredientes-chave -> {principais}")
        prompt = "\n".join([
            "Considere as seguintes receitas:",
            *resumo,
            "Sugira exatamente 3 drinks diferentes por receita, favorecendo harmonização com os ingredientes."
        ])
        obj = self.model_drinks.invoke([SystemMessage(content=self.system_drinks), HumanMessage(content=prompt)])
        return {"messages": [AIMessage(content=obj.model_dump_json(ensure_ascii=False, indent=2))]}

    def _hydrate_drinks(self, state: AgentState) -> dict:
        """Busca detalhes dos drinks sugeridos na API e monta o JSON final."""
        base_json = state["messages"][-2].content
        drinks_json = state["messages"][-1].content
        base = RespostaReceitasBase.model_validate_json(base_json)
        sugestoes = SugestoesDrinks.model_validate_json(drinks_json)
        
        mapa = {item.receita.lower(): item.drinks for item in sugestoes.sugestoes}
        receitas_finais = []
        
        for receita in base.receitas:
            nomes = mapa.get(receita.nome.lower(), [])[:3]
            drinks = [_build_drink_sugestao(nome) for nome in nomes]
            
            # Preenche até 3 drinks se necessário
            while len(drinks) < 3:
                drinks.append(DrinkSugestao(
                    nome=f"Drink sugerido {len(drinks)+1}",
                    ingredientes=[ItemIngrediente(nome="A definir", quantidade="a gosto")]
                ))
            
            receitas_finais.append(
                Receita(
                    nome=receita.nome,
                    ingredientes=receita.ingredientes,
                    modo_de_fazer=receita.modo_de_fazer,
                    rendimento=receita.rendimento,
                    sugestoes_drinks=drinks[:3]
                )
            )
        
        final = RespostaReceitas(receitas=receitas_finais)
        return {"messages": [AIMessage(content=final.model_dump_json(by_alias=True, ensure_ascii=False, indent=2))]}

In [15]:
def build_prompt(ingredientes: List[str], quantidade: int, restricoes: Optional[str] = None) -> str:
    """Constrói o prompt humano para o agente."""
    ing_lines = "\n".join(f"- {i}" for i in ingredientes)
    parts = [
        f"Ingredientes disponíveis:\n{ing_lines}",
        f"Quantidade de receitas: {quantidade}",
        (
            "Instruções: gere exatamente a quantidade de receitas solicitada, "
            "usando preferencialmente os ingredientes fornecidos. "
            "Permita sal, água, óleo e especiarias básicas. "
            "Se faltar algo essencial, proponha substituições plausíveis."
        )
    ]
    if restricoes:
        parts.append(f"Restrições/Preferências: {restricoes}")
    return "\n\n".join(parts)

### Referências de APIs adicionais

Explore alternativas à TheMealDB:

https://publicapis.io/alternatives/the-meal-db-api

In [16]:
# Exemplo de uso
ingredientes = [
    "massa (pasta)",
    "tomate",
    "manjericão",
    "alho",
    "queijo parmesão"
]
quantidade = 2
prompt_humano = build_prompt(ingredientes, quantidade)

agent = RecipeAgentGraph(model_tools="gpt-4o-mini", model_format="gpt-4o-mini")
resp = agent.graph.invoke({"messages": [HumanMessage(content=prompt_humano)]})

print(resp["messages"][-1].content)

{
  "receitas": [
    {
      "nome": "Pasta ao molho de tomate e manjericão",
      "ingredientes": [
        {
          "nome": "massa (pasta)",
          "quantidade": "250g"
        },
        {
          "nome": "tomate",
          "quantidade": "3 médios, picados"
        },
        {
          "nome": "manjericão",
          "quantidade": "folhas a gosto"
        },
        {
          "nome": "alho",
          "quantidade": "2 dentes, picados"
        },
        {
          "nome": "queijo parmesão",
          "quantidade": "a gosto, ralado"
        },
        {
          "nome": "azeite de oliva",
          "quantidade": "2 colheres de sopa"
        },
        {
          "nome": "sal",
          "quantidade": "a gosto"
        },
        {
          "nome": "pimenta",
          "quantidade": "a gosto"
        }
      ],
      "modo_de_fazer": [
        "Coe a massa em água salgada até ficar al dente. Reserve um pouco da água do cozimento.",
        "Em uma frigideira grande,